#  Energy Consumption Modeling for UAVs Based on Flight Data and Mission Parameters :
## 1. Data analysis and feature extraction


In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go


import plotly.express as px


In [2]:
df = pd.read_csv(r"C:\Users\raman\Desktop\Kanchan_Material\01_AirTransportAndLogistics\Course_Material\Semester_03\Research Task\12683453\flights.csv", delimiter=",", encoding="utf-8", header=0)
df.iloc[0:20, 0:28] 
#df.shape
print(df.columns.tolist())
df.iloc[100:150,3:7]

['flight', 'time', 'wind_speed', 'wind_angle', 'battery_voltage', 'battery_current', 'position_x', 'position_y', 'position_z', 'orientation_x', 'orientation_y', 'orientation_z', 'orientation_w', 'velocity_x', 'velocity_y', 'velocity_z', 'angular_x', 'angular_y', 'angular_z', 'linear_acceleration_x', 'linear_acceleration_y', 'linear_acceleration_z', 'speed', 'payload', 'altitude', 'date', 'time_day', 'route']


C:\Users\raman\AppData\Local\Temp\ipykernel_25596\4198744909.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\raman\Desktop\Kanchan_Material\01_AirTransportAndLogistics\Course_Material\Semester_03\Research Task\12683453\flights.csv", delimiter=",", encoding="utf-8", header=0)


,wind_angle,battery_voltage,battery_current,position_x
100,59.0,24.219669,0.111325,-79.782397
101,54.0,24.225929,0.111325,-79.782397
102,51.0,24.228434,0.111325,-79.782397
103,51.0,24.162075,0.914454,-79.782397
104,48.0,24.188368,0.405540,-79.782397
105,45.0,24.198383,0.381685,-79.782396
106,42.0,24.199636,0.389637,-79.782396
107,41.0,24.197132,0.405540,-79.782396
108,37.0,24.192123,0.453251,-79.782396
109,37.0,24.190872,0.524817,-79.782396


### Find Altitude, Vertical speed, Horizontal speed.

In [3]:
# Compute altitude relative to takeoff (up = positive)
df["altitude_measured"] = df["position_z"] - df.groupby("flight")["position_z"].transform("first")

# Compute vertical speed (m/s) from altitude_measured
df["vertical_speed"] = (
    df.groupby("flight")["altitude_measured"]
      .diff() / df.groupby("flight")["time"].diff()
)
print(df[df["flight"] == 1][["time", "altitude_measured", "vertical_speed"]].head(10))

df["vertical_speed_smooth"] = (
    df.groupby("flight")["vertical_speed"]
      .transform(lambda x: x.rolling(window=5, min_periods=1).mean())
)

# Compute horizontal speed (m/s)
df["horizontal_speed"] = np.sqrt(df["velocity_x"]**2 + df["velocity_y"]**2)

df["horizontal_speed_smooth"] = (
    df.groupby("flight")["horizontal_speed"]
      .transform(lambda x: x.rolling(window=5, min_periods=1).mean())
)

print(df[df["flight"] == 1][["time", "horizontal_speed"]].head(10))



   time  altitude_measured  vertical_speed
0  0.00           0.000000             NaN
1  0.20          -0.000346       -0.001728
2  0.30           0.000679        0.010252
3  0.50           0.002246        0.007835
4  0.60           0.003776        0.015297
5  0.81           0.003535       -0.001147
6  0.90           0.004929        0.015487
7  1.10           0.001680       -0.016246
8  1.20           0.000032       -0.016484
9  1.40           0.001752        0.008605
   time  horizontal_speed
0  0.00          0.014013
1  0.20          0.015496
2  0.30          0.016614
3  0.50          0.009389
4  0.60          0.010433
5  0.81          0.009317
6  0.90          0.009128
7  1.10          0.006518
8  1.20          0.004427
9  1.40          0.005595


### Find total acceralation, average power, peak power, Energy per flight

In [4]:
# df[["linear_acceleration_x", "linear_acceleration_y", "linear_acceleration_z"]].head(50)

# df["accel_total"] = np.sqrt(
#     df["linear_acceleration_x"]**2 +
#     df["linear_acceleration_y"]**2 +
#     df["linear_acceleration_z"]**2
# )
# df[["battery_current", "battery_voltage"]].head(50)


# 1. Instantaneous electrical power (make positive when discharging)
df["power_W"] = (df["battery_voltage"] * df["battery_current"])

# 2. Average and peak power
# avg_power = df.groupby("flight")["power_W"].mean()
# peak_power = df.groupby("flight")["power_W"].max()

# energy_per_flight_Wh = (
#     df.loc[:, ["flight", "time", "power_W"]]
#       .groupby("flight")[["time", "power_W"]]         # explicitly exclude 'flight'
#       .apply(lambda g: np.trapezoid(g["power_W"], g["time"]) / 3600.0)
#       .rename("energy_Wh")
# )

# df[["battery_current", "battery_voltage"]].head(100)


### Find different phase of flight using  altitude, horizontal and vertical speed. 

In [11]:

# Thresholds
vertical_threshold_up = 2.250    # m/s (positive means going up)
vertical_threshold_down = -0.7 # m/s (negative means going down)
altitude_threshold = 0.90       # m (to ensure actual height change)
hs_hover = 0.7                # m/s, low horizontal speed
hs_cruise = 1.05                # m/s, high horizontal speed

# Start with a default phase
df['phase'] = 'other'

# --- Climb: positive vertical speed + above ground ---

# Replace your climb mask with:
climb_mask = (
    (df['vertical_speed_smooth'] > 0.7) &
    (df['altitude_measured'] > altitude_threshold)
)
df.loc[climb_mask, 'phase'] = 'climb'

# --- Descent: negative vertical speed + above ground ---
descent_mask = (
    (df['vertical_speed_smooth'] < vertical_threshold_down) &
    (df['altitude_measured'] > altitude_threshold)
)
df.loc[descent_mask, 'phase'] = 'descent'

# --- Hover: nearly no vertical motion, low horizontal speed ---
hover_mask = (
    (df['vertical_speed_smooth'].abs() < 0.7) &
    (df['horizontal_speed_smooth'] < hs_hover) &
    (df['altitude_measured'] > altitude_threshold)
)
df.loc[hover_mask, 'phase'] = 'hover'

# --- Cruise: nearly no vertical motion, high horizontal speed ---
cruise_mask = (
    (df['vertical_speed_smooth'].abs() < 0.7) &
    # (df['vertical_speed_smooth'] < vertical_threshold_up) &
    (df['horizontal_speed_smooth'] > hs_cruise) &
    (df['altitude_measured'] > altitude_threshold)
)
df.loc[cruise_mask, 'phase'] = 'cruise'
#df.iloc[100:120, 0:35] 




### Visualize different phase of flight with Line graph

In [12]:


# ------------------ Select one flight ------------------
flight_id = 2
d = df[df["flight"] == flight_id].copy()

# ------------------ Define colors for phases ------------------
phase_colors = {
    "climb":   "rgba(0,255,0,0.25)",      # light green
    "descent": "rgba(255,0,0,0.25)",      # light red
    "hover":   "rgba(255,165,0,0.25)",    # light orange
    "cruise":  "rgba(0,0,255,0.25)",      # light blue
    "other":   "rgba(150,150,150,0.15)"   # light gray
}

# ------------------ Build customdata for hover ------------------
# customdata[:, 0] = altitude
# customdata[:, 1] = vertical speed
# customdata[:, 2] = horizontal speed
# customdata[:, 3] = phase
customdata = np.stack((
    d["altitude_measured"],
    d["vertical_speed_smooth"],
    d["horizontal_speed_smooth"],
    d["phase"]
), axis=-1)

# ------------------ Compute altitude range ------------------
y_min = d["altitude_measured"].min() - 1
y_max = d["altitude_measured"].max() + 1

fig = go.Figure()

# ------------------ 1) Add Altitude (left axis) ------------------
fig.add_trace(go.Scatter(
    x=d["time"],
    y=d["altitude_measured"],
    mode="lines",
    line=dict(width=2, color="blue"),
    name="Altitude (m)",
    customdata=customdata,
    hovertemplate=(
        "Time: %{x:.2f} s<br>"
        "Altitude: %{customdata[0]:.2f} m<br>"
        "Vertical speed: %{customdata[1]:.2f} m/s<br>"
        "Horizontal speed: %{customdata[2]:.2f} m/s<br>"
        "Phase: %{customdata[3]}<extra></extra>"
    )
))

# ------------------ 2) Add Vertical Speed (right axis) ------------------
fig.add_trace(go.Scatter(
    x=d["time"],
    y=d["vertical_speed_smooth"],
    mode="lines",
    line=dict(width=2, color="red"),
    name="Vertical speed (m/s)",
    yaxis="y2",
    customdata=customdata,
    hovertemplate=(
        "Time: %{x:.2f} s<br>"
        "Altitude: %{customdata[0]:.2f} m<br>"
        "Vertical speed: %{customdata[1]:.2f} m/s<br>"
        "Horizontal speed: %{customdata[2]:.2f} m/s<br>"
        "Phase: %{customdata[3]}<extra></extra>"
    )
))

# ------------------ 3) Add Phase Bands as Filled Traces ------------------
for phase, color in phase_colors.items():
    df_phase = d[d["phase"] == phase]
    if df_phase.empty:
        continue

    x_band = []
    y_band = []
    prev_idx = None
    start_t = None

    for idx, row in df_phase.iterrows():
        t = row["time"]
        if (prev_idx is None) or (idx != prev_idx + 1):
            if start_t is not None:
                # close previous rectangle
                x_band += [start_t, end_t, end_t, start_t, start_t, None]
                y_band += [y_min, y_min, y_max, y_max, y_min, None]
            start_t = t
        end_t = t
        prev_idx = idx

    # close last segment
    if start_t is not None:
        x_band += [start_t, end_t, end_t, start_t, start_t, None]
        y_band += [y_min, y_min, y_max, y_max, y_min, None]

    fig.add_trace(go.Scatter(
        x=x_band,
        y=y_band,
        mode="lines",
        fill="toself",
        fillcolor=color,
        line=dict(width=0),
        name=f"Phase: {phase}",
        hoverinfo="skip",  # don't show tooltip for background
        showlegend=True
    ))

# ------------------ 4) Layout ------------------
fig.update_layout(
    title=f"Flight {flight_id} – Altitude, Vertical Speed & Phases",
    xaxis=dict(title="Time (s)"),

    # Left Y-axis
    yaxis=dict(
        title=dict(text="Altitude (m)", font=dict(color="blue")),
        tickfont=dict(color="blue"),
        range=[y_min, y_max]
    ),

    # Right Y-axis
    yaxis2=dict(
        title=dict(text="Vertical speed (m/s)", font=dict(color="red")),
        tickfont=dict(color="red"),
        overlaying="y",
        side="right"
    ),

    template="plotly_white",
    legend=dict(x=1, y=1)
)
fig.update_layout(
    legend=dict(
        x=1.08,
        y=1,
        bgcolor="rgba(255,255,255,0.7)"
    ),
    margin=dict(r=120)
)


fig.show()


### Define Parameter

In [14]:
# -------------------------
# Global Parameters
# -------------------------

# Physical constants
rho = 1.225           # air density (kg/m³)
g = 9.81              # gravity (m/s²)

# Drone parameters
mass_kg = 2.50
disc_area_m2 = 0.22   # example rotor disc area
n_rotor = 4              # reference area
rotor_radius = 0.125   # [m] 10-inch propellers
eta_hover = 0.70        # hover efficiency (from ATM Table II for multicopter)

# Any other parameters:


### Find Hover Power

In [15]:
def hover_power():
    W = mass_kg * g
    A = n_rotor * np.pi * (rotor_radius ** 2)     # total rotor disc area
    DL = W / A
    P_hover = (W / eta_hover) * np.sqrt(DL / (2 * rho))
    return P_hover

P_hover = hover_power()
print(f"Hover Power = {P_hover/1000:.3f} kW")


Hover Power = 0.250 kW


In [7]:

# -----------------------------
# 1. UAV / Multicopter Parameters
# -----------------------------
m_uav = 2.50         # mass [kg]
g = 9.81             # gravity [m/s^2]
rho = 1.225          # air density [kg/m^3]

n_rotor = 4
rotor_radius = 0.125   # [m] 10-inch propellers

eta_hover = 0.70        # hover efficiency (from ATM Table II for multicopter)

# -----------------------------
# 2. Hover Power Using ATM Eq. (4)
# -----------------------------

# Total rotor area A
A = n_rotor * 2 * np.pi * rotor_radius**2

# Disc loading DL
W = m_uav * g
DL = W / A

# Hover power (Equation (4))
P_hover = (W / eta_hover) * np.sqrt(DL / (2 * rho))

print("=== Hover model from ATM Eq.(4) ===")
print("Total rotor disk area A =", A, "m^2")
print("Disc loading DL         =", DL, "N/m^2")
print("Hover power (model)     =", P_hover, "W")

# -----------------------------
# 3. Select HOVER samples from data
# -----------------------------
hover = df[df["phase"] == "hover"].copy()
print("\nRaw hover samples in df:", len(hover))

if hover.empty:
    print("No hover samples found in data.")
else:
    # -----------------------------
    # 4. CLEAN HOVER DATA
    #    - True hover = almost no vertical & horizontal motion
    #    - Remove ground / motors-off and crazy spikes
    # -----------------------------

    # You may need to adjust the column names if different
    if ("vertical_speed_smooth" in hover.columns) and ("horizontal_speed_smooth" in hover.columns):
        hover_clean = hover[
            (hover["vertical_speed_smooth"].abs() < 0.15) &
            (hover["horizontal_speed_smooth"] < 0.5)
        ].copy()
    else:
        print("Warning: vertical_speed_smooth / horizontal_speed_smooth not found. Using raw hover.")
        hover_clean = hover.copy()

    # Remove near-zero power (ground) and very high spikes
    hover_clean = hover_clean[
        (hover_clean["power_W"] > 150) & 
        (hover_clean["power_W"] < 600)
    ].copy()

    print("Cleaned hover samples:", len(hover_clean))

    if hover_clean.empty:
        print("No clean hover samples after filtering – check thresholds.")
    else:
        # -----------------------------
        # 5. Compute stats & errors
        # -----------------------------
        P_hover_data = hover_clean["power_W"].values

        P_hover_data_mean = P_hover_data.mean()
        P_hover_data_std  = P_hover_data.std()

        print("\n=== Hover data (cleaned) ===")
        print(f"Actual hover power (mean) = {P_hover_data_mean:.1f} W")
        print(f"Actual hover power (std)  = {P_hover_data_std:.1f} W")

        # Constant model = P_hover for all hover samples
        P_hover_model = np.full_like(P_hover_data, P_hover)

        # MAE & RMSE
        MAE  = np.mean(np.abs(P_hover_data - P_hover_model))
        RMSE = np.sqrt(np.mean((P_hover_data - P_hover_model)**2))

        rel_error_mean = (P_hover_data_mean - P_hover) / P_hover_data_mean * 100.0

        print("\n=== Hover model vs data (cleaned) ===")
        print(f"Hover power (model)       = {P_hover:.1f} W")
        print(f"Difference (mean data - model) = {P_hover_data_mean - P_hover:.1f} W")
        print(f"Relative error (mean)          = {rel_error_mean:.1f} %")
        print(f"MAE  = {MAE:.1f} W")
        print(f"RMSE = {RMSE:.1f} W")


=== Hover model from ATM Eq.(4) ===
Total rotor disk area A = 0.39269908169872414 m^2
Disc loading DL         = 62.45239966925974 N/m^2
Hover power (model)     = 176.88968112021686 W

Raw hover samples in df: 21108
Cleaned hover samples: 2659

=== Hover data (cleaned) ===
Actual hover power (mean) = 488.5 W
Actual hover power (std)  = 76.0 W

=== Hover model vs data (cleaned) ===
Hover power (model)       = 176.9 W
Difference (mean data - model) = 311.6 W
Relative error (mean)          = 63.8 %
MAE  = 311.9 W
RMSE = 320.8 W


### Find Climb Power

In [8]:
# Is climb is pure Vertical or not?
climb_raw = df[df["phase"]=="climb"]
print(climb_raw["vertical_speed_smooth"].describe())
print(climb_raw["horizontal_speed_smooth"].describe())



# ==========================================
# VERTICAL CLIMB POWER USING ATM PAPER EQ.
# ==========================================



# ------------------------------------------
# 1. UAV / rotor parameters (same as hover)
# ------------------------------------------
m_uav = 2.50         # [kg]
g = 9.81             # [m/s^2]
rho = 1.225          # [kg/m^3]

n_rotor = 4
rotor_radius = 0.125   # [m]
eta_hover = 0.50        # hover efficiency (from paper / your choice)

# Total rotor area and weight
A = n_rotor * np.pi * rotor_radius**2
W = m_uav * g

# Disc loading and hover induced velocity
DL = W / A
v_h = np.sqrt(W / (2.0 * rho * A))   # hover induced velocity [m/s]

# Hover power from paper (you already used this)
P_hover = (W / eta_hover) * np.sqrt(DL / (2.0 * rho))

print("W (N)    =", W)
print("A (m^2)  =", A)
print("DL       =", DL, "N/m^2")
print("v_h      =", v_h, "m/s")
print("P_hover  =", P_hover, "W")

# ------------------------------------------
# 2. Select CLIMB data (same idea as before)
# ------------------------------------------
climb_df = df[df["phase"] == "climb"].copy()

# Keep only real upward climbs (tune threshold as needed)
# climb_df = climb_df[climb_df["vertical_speed_smooth"] > 2.5]

# Filter unrealistic power outliers (tune if needed)
# climb_df = climb_df[(climb_df["power_W"] > 50) & (climb_df["power_W"] < 500)]

# Drop NaNs
climb_df = climb_df.dropna(subset=["vertical_speed_smooth", "power_W"])

print("Number of climb samples after filtering:", len(climb_df))

if climb_df.empty:
    print("No climb samples available after filtering - cannot evaluate climb model.")
else:
    # ------------------------------------------
    # 3. Vertical climb power from ATM paper
    #    P_vc = P_hover * ( RoC/v_h + v_i/v_h )
    #    v_i/v_h = RoC/(2v_h) + sqrt( (RoC/(2v_h))^2 + 1 )
    # ------------------------------------------

    def power_vertical_climb_paper(RoC, P_hover, v_h):
        """
        RoC : vertical rate of climb [m/s], positive
        P_hover : hover power [W]
        v_h : hover induced velocity [m/s]
        Returns: P_vertical_climb [W]
        """
        RoC = np.asarray(RoC)
        eps = 1e-9  # avoid division by zero
        v_ratio = RoC / (2.0 * (v_h + eps))
        vi_over_vh = - v_ratio + np.sqrt(v_ratio**2 + 1.0)
        return P_hover * (RoC / (v_h + eps) + vi_over_vh)

    # Extract arrays
    ROC = climb_df["vertical_speed_smooth"].values   # [m/s]
    P_measured = climb_df["power_W"].values         # [W]

    # Model prediction from paper equation
    P_climb_model = power_vertical_climb_paper(ROC, P_hover, v_h)

    # ------------------------------------------
    # 4. Error metrics (MAE, RMSE, % errors)
    # ------------------------------------------
    errors = P_measured - P_climb_model
    MAE = np.mean(np.abs(errors))
    RMSE = np.sqrt(np.mean(errors**2))

    mean_climb_power = P_measured.mean()
    MAE_percent = (MAE / mean_climb_power) * 100.0
    RMSE_percent = (RMSE / mean_climb_power) * 100.0

    print("\n=== Vertical climb model (ATM paper) ===")
    print("Mean measured climb power  =", mean_climb_power, "W")
    print("MAE                         =", MAE, "W")
    print("RMSE                        =", RMSE, "W")
    print("MAE  [% of mean power]      =", MAE_percent, "%")
    print("RMSE [% of mean power]      =", RMSE_percent, "%")

    # ------------------------------------------
    # 5. Optional: summary table for your report
    # ------------------------------------------
    climb_summary = pd.DataFrame({
        "mass_kg": [m_uav],
        "weight_N": [W],
        "P_hover_W": [P_hover],
        "v_h_mps": [v_h],
        "N_climb_samples": [len(climb_df)],
        "mean_climb_P_W": [mean_climb_power],
        "MAE_W": [MAE],
        "RMSE_W": [RMSE],
        "MAE_percent": [MAE_percent],
        "RMSE_percent": [RMSE_percent],
    })

    print("\nClimb model summary (paper-based):")
    print(climb_summary)


count    28765.000000
mean         2.931216
std          0.763219
min          0.700039
25%          2.711820
50%          3.093606
75%          3.381659
max          5.380489
Name: vertical_speed_smooth, dtype: float64
count    28765.000000
mean         0.458742
std          1.432752
min          0.007149
25%          0.076715
50%          0.123498
75%          0.207881
max         12.239706
Name: horizontal_speed_smooth, dtype: float64
W (N)    = 24.525000000000002
A (m^2)  = 0.19634954084936207
DL       = 124.90479933851948 N/m^2
v_h      = 7.140136605865192 m/s
P_hover  = 350.22370051768775 W
Number of climb samples after filtering: 28765

=== Vertical climb model (ATM paper) ===
Mean measured climb power  = 597.0950650273331 W
MAE                         = 170.90362811241343 W
RMSE                        = 189.08429468860461 W
MAE  [% of mean power]      = 28.622515596338083 %
RMSE [% of mean power]      = 31.667368525303242 %

Climb model summary (paper-based):
   mass_kg  weight

### Find Cruise Power

In [153]:


# --------------------------------------------------
# 1. Assumptions / prerequisites
# --------------------------------------------------
# - df exists
# - df["phase"], df["horizontal_speed_smooth"], df["vertical_speed_smooth"], df["power_W"] exist
# - P_hover already computed from hover equation (≈ 250 W for M100)

print("Using P_hover =", P_hover, "W")

# --------------------------------------------------
# 2. Select CRUISE samples (same filter as before)
# --------------------------------------------------
cruise = df[df["phase"] == "cruise"].copy()

# Nearly level flight
cruise = cruise[cruise["vertical_speed_smooth"].abs() < 0.3]

# Reasonable forward speed band
cruise = cruise[(cruise["horizontal_speed_smooth"] > 3.0) &
                (cruise["horizontal_speed_smooth"] < 12.0)]

# Filter power outliers
cruise = cruise[(cruise["power_W"] > 50) & (cruise["power_W"] < 800)]

cruise = cruise.dropna(subset=["horizontal_speed_smooth", "power_W"])

print("Filtered cruise samples:", len(cruise))

if cruise.empty:
    print("No cruise samples available after filtering – cannot fit model.")
else:
    # --------------------------------------------------
    # 3. Prepare regression: P(V) = P_hover + k2 V^2 + k3 V^3
    #    => y = P - P_hover = k2 V^2 + k3 V^3
    # --------------------------------------------------
    V = cruise["horizontal_speed_smooth"].values   # [m/s]
    P = cruise["power_W"].values                   # [W]

    y = P - P_hover                                # extra power above hover
    X = np.column_stack([V**2, V**3])              # [N, 2] -> columns: V^2, V^3

    # Solve least squares: X @ [k2, k3] ≈ y
    coeffs, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)
    k2, k3 = coeffs

    print("\n=== Fitted multicopter cruise model ===")
    print("k2 (W / m^2/s^2) =", k2)
    print("k3 (W / m^3/s^3) =", k3)

    # --------------------------------------------------
    # 4. Compute modeled power and errors
    # --------------------------------------------------
    P_model = P_hover + (k2 * V**2 + k3 * V**3)

    errors = P - P_model
    MAE  = np.mean(np.abs(errors))
    RMSE = np.sqrt(np.mean(errors**2))
    mean_P = P.mean()

    MAE_pct  = MAE  / mean_P * 100.0
    RMSE_pct = RMSE / mean_P * 100.0

    print("\n=== Cruise model performance (P = P_hover + k2 V^2 + k3 V^3) ===")
    print("Mean measured cruise power =", mean_P, "W")
    print("MAE  =", MAE, "W")
    print("RMSE =", RMSE, "W")
    print("MAE %  =", MAE_pct)
    print("RMSE % =", RMSE_pct)

    # --------------------------------------------------
    # 5. (Optional) store model power in cruise df
    # --------------------------------------------------
    cruise["P_cruise_multicopter_model"] = P_model

    # If you want to merge back into main df:
    df.loc[cruise.index, "P_cruise_multicopter_model"] = cruise["P_cruise_multicopter_model"]


Using P_hover = 250.1597860840627 W
Filtered cruise samples: 72801

=== Fitted multicopter cruise model ===
k2 (W / m^2/s^2) = 15.041425063045812
k3 (W / m^3/s^3) = -1.236228354415657

=== Cruise model performance (P = P_hover + k2 V^2 + k3 V^3) ===
Mean measured cruise power = 505.2724010532795 W
MAE  = 87.26888604949725 W
RMSE = 106.64704444818449 W
MAE %  = 17.271651067340805
RMSE % = 21.10684142372915


### Find Descent Power

In [155]:
# Extract descent samples
descent = df[df["phase"] == "descent"].copy()

if len(descent) == 0:
    print("No descent samples found in data.")
else:
    # Compute averages for decision
    mean_vz = descent["vertical_speed_smooth"].mean()
    mean_vh = descent["horizontal_speed_smooth"].mean()
    
    print("=== DESCENT DIAGNOSTICS FROM DATA ===")
    print(f"Mean vertical speed (vz): {mean_vz:.3f} m/s")
    print(f"Mean horizontal speed (vh): {mean_vh:.3f} m/s")
    print("--------------------------------------")
    
    # Decision logic
    if mean_vh < 1.0 and mean_vz < -0.5:
        print("Detected: PURE VERTICAL DESCENT")
        print("Recommended equation: ATM Eq.(18) -- Vertical Descent Power")
    elif mean_vh >= 1.0:
        print("Detected: FORWARD DESCENT")
        print("Recommended equation: Cruise Power (Eq.16)")
    else:
        print("Mixed descent characteristics.")
        print("Suggested: Split into vertical-descent and forward-descent segments.")

 # Correct descent model for multicopter: P_desc = k_desc * P_hover

P_hover = 250.1597860840627  # your previously computed hover power

descent = df[df["phase"] == "descent"].copy()

# only valid downward motion
descent = descent[descent["vertical_speed_smooth"] < -0.3]
descent = descent[(descent["power_W"] > 50) & (descent["power_W"] < 800)]

P_desc_data = descent["power_W"].values

# estimate k_desc
k_desc = P_desc_data.mean() / P_hover

print("Estimated k_desc =", k_desc)

# model prediction
P_desc_model = k_desc * P_hover
P_desc_model_array = np.full_like(P_desc_data, P_desc_model)

# errors
MAE = np.mean(np.abs(P_desc_data - P_desc_model_array))
RMSE = np.sqrt(np.mean((P_desc_data - P_desc_model_array)**2))

print("Mean descent power (data) =", P_desc_data.mean())
print("Descent model power       =", P_desc_model)
print("MAE =", MAE)
print("RMSE =", RMSE)
print("MAE % =", MAE / P_desc_data.mean() * 100)
print("RMSE % =", RMSE / P_desc_data.mean() * 100)       



=== DESCENT DIAGNOSTICS FROM DATA ===
Mean vertical speed (vz): -1.802 m/s
Mean horizontal speed (vh): 0.374 m/s
--------------------------------------
Detected: PURE VERTICAL DESCENT
Recommended equation: ATM Eq.(18) -- Vertical Descent Power
Estimated k_desc = 2.015844458700139
Mean descent power (data) = 504.28321856716997
Descent model power       = 504.28321856716997
MAE = 53.6609753844015
RMSE = 70.66810815489262
MAE % = 10.641039282819982
RMSE % = 14.013575219830502


In [156]:
# ======================================================
# STEP 2 — Combined Physics-Based Power Model (Task 2)
# ======================================================

def physics_power_model(row):
    phase = row["phase"]
    Vh = row["horizontal_speed_smooth"]
    Vz = row["vertical_speed_smooth"]
    
    # -----------------------------------
    # 1) HOVER
    # -----------------------------------
    if phase == "hover":
        return P_hover
    
    # -----------------------------------
    # 2) CLIMB (ATM vertical climb)
    # -----------------------------------
    if phase == "climb":
        return power_vertical_climb_paper(Vz, P_hover, v_h)
    
    # -----------------------------------
    # 3) CRUISE (V² + V³ multicopter model)
    # -----------------------------------
    if phase == "cruise":
        return P_hover + k2 * (Vh**2) + k3 * (Vh**3)
    
    # -----------------------------------
    # 4) DESCENT (Option B — empirical)
    # -----------------------------------
    if phase == "descent":
        return k_desc * P_hover
    
    # -----------------------------------
    # 5) UNKNOWN PHASE → no model
    # -----------------------------------
    return np.nan


# APPLY MODEL TO ENTIRE DATASET
df["P_phys_model"] = df.apply(physics_power_model, axis=1)

print("Step 2 complete: df['P_phys_model'] created.")
print(df[["phase", "power_W", "P_phys_model"]].head())







Step 2 complete: df['P_phys_model'] created.
   phase   power_W  P_phys_model
0  other  2.118702           NaN
1  other  2.311789           NaN
2  other  2.311669           NaN
3  other  2.311550           NaN
4  other  1.925196           NaN


In [157]:


# Keep only rows where we actually have a model (drop 'other' etc.)
df_model = df.dropna(subset=["P_phys_model", "power_W"]).copy()

results = {}

for ph in ["hover", "climb", "cruise", "descent"]:
    d = df_model[df_model["phase"] == ph]
    if len(d) == 0:
        continue
    
    P_data  = d["power_W"].values
    P_model = d["P_phys_model"].values
    
    MAE  = np.mean(np.abs(P_data - P_model))
    RMSE = np.sqrt(np.mean((P_data - P_model)**2))
    mean_P = P_data.mean()
    
    results[ph] = {
        "N_samples": len(d),
        "mean_power_W": mean_P,
        "MAE_W": MAE,
        "RMSE_W": RMSE,
        "MAE_percent": MAE / mean_P * 100.0,
        "RMSE_percent": RMSE / mean_P * 100.0,
    }

# Also overall (all modeled phases together)
P_data_all  = df_model["power_W"].values
P_model_all = df_model["P_phys_model"].values

MAE_all  = np.mean(np.abs(P_data_all - P_model_all))
RMSE_all = np.sqrt(np.mean((P_data_all - P_model_all)**2))
mean_P_all = P_data_all.mean()

results["ALL_PHASES"] = {
    "N_samples": len(df_model),
    "mean_power_W": mean_P_all,
    "MAE_W": MAE_all,
    "RMSE_W": RMSE_all,
    "MAE_percent": MAE_all / mean_P_all * 100.0,
    "RMSE_percent": RMSE_all / mean_P_all * 100.0,
}

results_df = pd.DataFrame(results).T
print(results_df)


            N_samples  mean_power_W       MAE_W      RMSE_W  MAE_percent  \
hover         21108.0    308.367996  263.996259  274.032502    85.610784   
climb         27449.0    600.782642  188.603395  206.692446    31.392950   
cruise        96501.0    511.088205  101.294515  128.341434    19.819380   
descent       45055.0    504.939125   54.229306   72.188567    10.739771   
ALL_PHASES   190113.0    500.073480  120.810944  155.247427    24.158638   

            RMSE_percent  
hover          88.865416  
climb          34.403864  
cruise         25.111406  
descent        14.296489  
ALL_PHASES     31.044923  
